# Limpieza y Transformación de Datos

En este notebook, realizaremos la limpieza y transformación de los datasets proporcionados siguiendo las reglas formales especificadas. Cada regla será explicada en detalle antes de su implementación en el código.

## Regla 0: Corrección de nombres de columnas

En esta regla, renombraremos las columnas de los datasets para que sean más descriptivas y consistentes. Los cambios son los siguientes:

- `id` → `customer_id`
- `customer_id` → `transaction_id`

In [1]:
import pandas as pd
import numpy as np

# Cargar los datasets
customer_data = pd.read_csv(r"csv\customer_data.csv")
retail_data = pd.read_csv(r"csv\retail_data.csv")

customer_og = customer_data.copy()
retail_og = retail_data.copy()

# Renombrar columnas
retail_data.rename(columns={'customer_id': 'transaction_id'}, inplace=True)
retail_data.rename(columns={'id': 'customer_id'}, inplace=True)

In [2]:
# Verificar los cambios
retail_data.head()

,customer_id,purchase_date,product_category,amount,transaction_id
0,230,2025-05-11,Clothing,181.73,1034
1,630,2025-06-19,Electronics,442.32,1020
2,867,2025-02-16,Home & Kitchen,263.12,1049
3,763,2025-02-13,Home & Kitchen,434.05,1058
4,758,2025-04-15,Clothing,489.6,1077


In [3]:
customer_data.head()

,id,full_name,email,phone,address,signup_date,name,gender,age
0,1,Kara Kim,kara.kim1924@yahoo.com,804-681-7662,"8971 Pine Ave, Greenville, TX 76366",2022-03-13,Elena,Female,60
1,2,Kelly Peterson,NaN,(467) 700-2147,"7619 Cedar Ln, Madison, CA 32004",10/04/2019,Bob,Female,50
2,3,Henry Martin,henry.martin0365@mail.com,(378) 615-9326,"785 Elm St, Centerville, FL 70040",01/11/2019,Carlos,Male,36
3,4,Walter Evans,walter.evans9757@hotmail.com,406-811-1412,"9383 Elm St, Springfield, TX 27467",09/22/2021,Diana,Female,64
4,5,William Anderson,william.anderson8799@gmail.com,260-206-8340,"615 Sunset Blvd, Riverside, IL 02711",11/15/2020,George,Female,29


## Regla 1: Ids únicos y no nulos

En esta regla, nos aseguraremos de que la columna id sea única y no contenga valores nulos. Si hay valores nulos, se asignará un número entero que no esté usado en la secuencia. Si hay duplicados, se conservará el primer registro y se reasignarán nuevos IDs disponibles a los duplicados.

In [4]:
# Asegurar que id sea único y no nulo
existing_ids = set(customer_data['id'].dropna())
next_id = max(existing_ids) + 1 if existing_ids else 1

# Reemplazar valores nulos en id
for idx, row in customer_data.iterrows():
    if pd.isna(row['id']):
        while next_id in existing_ids:
            next_id += 1
        customer_data.at[idx, 'id'] = next_id
        existing_ids.add(next_id)

# Eliminar duplicados conservando el primer registro y reasignar IDs únicos a los duplicados
customer_data = customer_data.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)

# Verificar los cambios
print("Customer IDs únicos y no nulos:", customer_data['id'].is_unique)
print("Customer IDs nulos:", customer_data['id'].isnull().sum())
print("Total registros:", len(customer_data))
customer_data[['id']]

Customer IDs únicos y no nulos: True
Customer IDs nulos: 0
Total registros: 1000


,id
0,1
1,2
2,3
3,4
4,5
...,...
995,996
996,997
997,998
998,999


## Retail

nos aseguraremos de que la columnas IDS sean únicas y no contenga valores nulos. Si hay valores nulos, se asignará un número entero que no esté usado en la secuencia. Si hay duplicados, se conservará el primer registro y se reasignarán nuevos IDs disponibles a los duplicados.

### transaction_id

In [5]:
# Asegurar que transaction_id sea único y no nulo
existing_ids = set(retail_data['transaction_id'].dropna())
next_id = max(existing_ids) + 1 if existing_ids else 1

# Reemplazar valores nulos en transaction_id
for idx, row in retail_data.iterrows():
    if pd.isna(row['transaction_id']):
        while next_id in existing_ids:
            next_id += 1
        retail_data.at[idx, 'transaction_id'] = next_id
        existing_ids.add(next_id)

# Reasignar IDs únicos a los duplicados
for idx, row in retail_data.iterrows():
    if retail_data.duplicated(subset=['transaction_id']).iloc[idx]:
        while next_id in existing_ids:
            next_id += 1
        retail_data.at[idx, 'transaction_id'] = next_id
        existing_ids.add(next_id)

# Verificar los cambios
print("Transaction IDs únicos y no nulos:", retail_data['transaction_id'].is_unique)
print("Transaction IDs nulos:", retail_data['transaction_id'].isnull().sum())
retail_data[['transaction_id']]

Transaction IDs únicos y no nulos: True
Transaction IDs nulos: 0


,transaction_id
0,1034
1,1020
2,1049
3,1058
4,1077
...,...
815,1815
816,1816
817,1817
818,1818


## Regla 2: purchase_date y signup_date

En esta regla, convertiremos las columnas `purchase_date` y `signup_date` al tipo datetime. Validaremos que las fechas estén en el rango permitido (mayores a 2000-01-01 y menores a la fecha actual). Las fechas inválidas serán convertidas a `NaT`.

In [6]:
from datetime import datetime

# Definir los límites de las fechas
cutoff_min = datetime(2000, 1, 1)
cutoff_max = datetime.now()

# Función para estandarizar formatos de fecha
def standardize_date(date):
    for fmt in ("%m/%d/%Y", "%Y-%m-%d", "%d-%m-%Y"):
        try:
            return datetime.strptime(date, fmt).strftime("%Y-%m-%d")
        except ValueError:
            continue
    return None

# Convertir las columnas a datetime y validar el rango
def validate_dates(df, column):
    df[column] = df[column].apply(lambda x: standardize_date(x) if pd.notna(x) else None)
    df[column] = pd.to_datetime(df[column], errors='coerce')
    df.loc[(df[column] < cutoff_min) | (df[column] > cutoff_max), column] = pd.NaT

validate_dates(customer_data, 'signup_date')
validate_dates(retail_data, 'purchase_date')

# Verificar los cambios
print("Fechas válidas en signup_date:", customer_data['signup_date'].notna().sum())
print("Fechas válidas en purchase_date:", retail_data['purchase_date'].notna().sum())
customer_data[['signup_date']].info()

Fechas válidas en signup_date: 902
Fechas válidas en purchase_date: 796
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   signup_date  902 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.9 KB


## Regla 3: sales_amount

En esta regla, transformaremos la columna `sales_amount` para que sea numérica. Los valores negativos o no válidos serán reemplazados por `NaN`. Posteriormente, los valores faltantes o inválidos serán imputados utilizando la media de la columna agrupada por `product_category`.

In [7]:
# Convertir amount a numérico y manejar valores inválidos
retail_data['amount'] = pd.to_numeric(retail_data['amount'], errors='coerce')

# Reemplazar valores negativos por NaN
retail_data.loc[retail_data['amount'] < 0, 'amount'] = pd.NA

# Imputar valores faltantes usando la media agrupada por product_category
means = retail_data.groupby('product_category')['amount'].transform('mean')
retail_data['amount'] = retail_data['amount'].fillna(means)

# Verificar los cambios
print("Valores nulos en amount:", retail_data['amount'].isna().sum())
retail_data[['amount']]

Valores nulos en amount: 2


,amount
0,181.73
1,442.32
2,263.12
3,434.05
4,489.60
...,...
815,61.89
816,74.31
817,33.54
818,371.75


## Regla 4: email

En esta regla, transformaremos los correos electrónicos a minúsculas y validaremos su formato utilizando una expresión regular. Los correos que no cumplan con el patrón serán marcados como `NaN` o eliminados.

In [8]:
import re

# Definir el patrón de validación para correos electrónicos
EMAIL_REGEX = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

# Convertir correos a minúsculas y validar
customer_data['email'] = customer_data['email'].str.lower()
customer_data['email'] = customer_data['email'].apply(lambda x: x if re.match(EMAIL_REGEX, str(x)) else pd.NA)

# Verificar los cambios
print("Correos válidos:", customer_data['email'].notna().sum())

Correos válidos: 892


## Regla 5: phone_number

En esta regla, validaremos que los números de teléfono contengan exactamente 10 dígitos numéricos. Eliminaremos paréntesis y otros caracteres no numéricos. Si el número no tiene exactamente 10 dígitos, se eliminará el registro.

In [9]:
# Validar y limpiar números de teléfono
customer_data['phone'] = customer_data['phone'].str.replace(r'^\+1', '', regex=True)  # Eliminar prefijo +1
customer_data['phone'] = customer_data['phone'].str.replace(r'[^\d]', '', regex=True)  # Eliminar símbolos y espacios
customer_data['phone'] = customer_data['phone'].apply(lambda x: x if len(str(x)) == 10 else pd.NA)

# Verificar los cambios
print("Números de teléfono válidos:", customer_data['phone'].notna().sum())
customer_data['phone']

Números de teléfono válidos: 902


0      8046817662
1      4677002147
2      3786159326
3      4068111412
4      2602068340
          ...    
995    7429909163
996    9059943879
997    4102043172
998    6092871080
999    3453876657
Name: phone, Length: 1000, dtype: object

## Regla 6: age

En esta regla, transformaremos la columna `age` para que sea numérica. Validaremos que los valores sean mayores o iguales a 18 y menores a 100. Si algún valor está fuera de este rango, será modificado a 18.

In [10]:
customer_data['age'] = pd.to_numeric(customer_data['age'], errors='coerce')

# Reemplazar valores fuera del rango [15, 90] por 90 (manteniendo NaN sin cambios)
customer_data['age'] = np.where(
    (customer_data['age'] < 15) | (customer_data['age'] > 90),
    90,
    customer_data['age']
)

# Verificar los cambios
print("Total de edades en rango [15, 90]:", customer_data['age'].between(15, 90).sum())
print("Edades transformadas a 90:", (customer_data['age'] == 90).sum())

Total de edades en rango [15, 90]: 1000
Edades transformadas a 90: 13


## Resumen Final

En esta sección, generaremos un resumen final que incluirá:

1. El número de registros modificados por cada regla.
2. La forma final del DataFrame (`df.shape`).

In [11]:
customer_data.head()

,id,full_name,email,phone,address,signup_date,name,gender,age
0,1,Kara Kim,kara.kim1924@yahoo.com,8046817662,"8971 Pine Ave, Greenville, TX 76366",2022-03-13,Elena,Female,60
1,2,Kelly Peterson,<NA>,4677002147,"7619 Cedar Ln, Madison, CA 32004",2019-10-04,Bob,Female,50
2,3,Henry Martin,henry.martin0365@mail.com,3786159326,"785 Elm St, Centerville, FL 70040",2019-01-11,Carlos,Male,36
3,4,Walter Evans,walter.evans9757@hotmail.com,4068111412,"9383 Elm St, Springfield, TX 27467",2021-09-22,Diana,Female,64
4,5,William Anderson,william.anderson8799@gmail.com,2602068340,"615 Sunset Blvd, Riverside, IL 02711",2020-11-15,George,Female,29


In [12]:
customer_og.head()

,id,full_name,email,phone,address,signup_date,name,gender,age
0,1,Kara Kim,kara.kim1924@yahoo.com,804-681-7662,"8971 Pine Ave, Greenville, TX 76366",2022-03-13,Elena,Female,60
1,2,Kelly Peterson,NaN,(467) 700-2147,"7619 Cedar Ln, Madison, CA 32004",10/04/2019,Bob,Female,50
2,3,Henry Martin,henry.martin0365@mail.com,(378) 615-9326,"785 Elm St, Centerville, FL 70040",01/11/2019,Carlos,Male,36
3,4,Walter Evans,walter.evans9757@hotmail.com,406-811-1412,"9383 Elm St, Springfield, TX 27467",09/22/2021,Diana,Female,64
4,5,William Anderson,william.anderson8799@gmail.com,260-206-8340,"615 Sunset Blvd, Riverside, IL 02711",11/15/2020,George,Female,29


In [13]:
retail_data

,customer_id,purchase_date,product_category,amount,transaction_id
0,230,2025-05-11,Clothing,181.73,1034
1,630,2025-06-19,Electronics,442.32,1020
2,867,2025-02-16,Home & Kitchen,263.12,1049
3,763,2025-02-13,Home & Kitchen,434.05,1058
4,758,2025-04-15,Clothing,489.60,1077
...,...,...,...,...,...
815,494,2025-04-25,Clothing,61.89,1815
816,555,2025-03-04,Sports,74.31,1816
817,491,2025-03-14,Toys,33.54,1817
818,13,2025-06-19,Electronics,371.75,1818


In [14]:
retail_og

,id,purchase_date,product_category,amount,customer_id
0,230,2025-05-11,Clothing,181.73,1034
1,630,2025-06-19,Electronics,442.32,1020
2,867,2025-02-16,Home & Kitchen,263.12,1049
3,763,2025-02-13,Home & Kitchen,434.05,1058
4,758,2025-04-15,Clothing,489.6,1077
...,...,...,...,...,...
815,494,2025-04-25,Clothing,61.89,1065
816,555,2025-03-04,Sports,74.31,1061
817,491,2025-03-14,Toys,33.54,1096
818,13,2025-06-19,Electronics,371.75,1070


In [15]:
retail_data.info(), customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       820 non-null    int64         
 1   purchase_date     796 non-null    datetime64[ns]
 2   product_category  796 non-null    object        
 3   amount            818 non-null    float64       
 4   transaction_id    820 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 32.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           1000 non-null   int64         
 1   full_name    916 non-null    object        
 2   email        892 non-null    object        
 3   phone        902 non-null    object        
 4   address      897 non-null    obj

(None, None)

In [16]:
import os

output_folder = "data_clean"
os.makedirs(output_folder, exist_ok=True)

# Guardar los archivos CSV
customer_data.to_csv(os.path.join(output_folder, "customer_data_clean.csv"))
retail_data.to_csv(os.path.join(output_folder, "retail_data_clean.csv"))

print("Archivos limpios guardados en la carpeta 'data_clean'.")

Archivos limpios guardados en la carpeta 'data_clean'.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8827a6c5-c255-41ba-9603-9339500bd604' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>